In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
config = {
    'domain': 'drink',
    'postgres': False,
    'gen_model_search_space': 1,
    'disc_model_search_space': 2,
    'disc_params_search': {
        'lr'        : [1, 50],
        'max_steps' : [2, 3]
    },
    'print_freq': 1,
    'download_data': True,
    'supervision': 'generative',
}

In [3]:
# Get DB connection string and add to globals
# NOTE: $SNORKELDB must be set before any snorkel imports
import os

default_db_name = 'babble_' + config['domain'] + ('_debug' if config.get('debug', False) else '')
DB_NAME = config.get('db_name', default_db_name)
if 'postgres' in config and config['postgres']:
    DB_TYPE = 'postgres'
else:
    DB_TYPE = 'sqlite'
    DB_NAME += '.db'
DB_ADDR = "localhost:{0}".format(config['db_port']) if 'db_port' in config else ""
os.environ['SNORKELDB'] = '{0}://{1}/{2}'.format(DB_TYPE, DB_ADDR, DB_NAME)
print("$SNORKELDB = {0}".format(os.environ['SNORKELDB']))

$SNORKELDB = sqlite:///babble_drink.db


In [4]:
from snorkel import SnorkelSession
session = SnorkelSession()

# Resolve config conflicts (nb_config > local_config > global_config)
from snorkel.contrib.babble.pipelines import merge_configs, get_local_pipeline
config = merge_configs(config)

if config['debug']:
    print("NOTE: --debug=True: modifying parameters...")
    config['max_docs'] = 100
    config['gen_model_search_space'] = 2
    config['disc_model_search_space'] = 2
    config['gen_params_default']['epochs'] = 25
    config['disc_params_default']['n_epochs'] = 5

from snorkel.models import candidate_subclass
candidate_class = candidate_subclass(config['candidate_name'], config['candidate_entities'])

pipeline = get_local_pipeline(config['domain'])
pipe = pipeline(session, candidate_class, config)

Overwriting download_data=False to download_data=True
Overwriting max_steps=[3, 5] to max_steps=[2, 3]
Overwriting domain=None to domain=drink
Overwriting disc_model_search_space=10 to disc_model_search_space=2
Overwriting gen_model_search_space=10 to gen_model_search_space=1
Overwriting splits=[0, 1, 2] to splits=[0, 1]
Overwriting disc_model_class=lstm to disc_model_class=inception_v3


In [ ]:
%time pipe.parse()

In [ ]:
%time pipe.extract()

In [ ]:
%time pipe.load_gold()

In [ ]:
%time pipe.collect()

In [ ]:
%time pipe.label()

In [ ]:
%time pipe.supervise()

In [ ]:
%time pipe.classify()